In [ ]:
!pip install geopandas
!pip install geemap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.0 MB 8.6 MB/s 
     |████████████████████████████████| 16.7 MB 33.1 MB/s 
     |████████████████████████████████| 6.3 MB 38.6 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.1 MB 23.8 MB/s 
     |████████████████████████████████| 1.2 MB 48.9 MB/s 
     |████████████████████████████████| 1.3 MB 56.5 MB/s 
     |████████████████████████████████| 8.8 MB 54.1 MB/s 
     |████████████████████████████████| 198 kB 64.3 MB/s 
     |████████████████████████████████| 99 kB 6.8 MB/s 
     |████████████████████████████████| 55 kB 3.5 MB/s 
     |████████████████████████████████| 3.7 MB 51.4 MB/s 
     |████████████████████████████████| 98 kB 6.7 MB/s 
     |████████████████████████████████| 3.0 MB 48.1 MB/s 
     |████████████████████████████████| 46 kB 3.7 MB/s

In [ ]:
import ee
import geemap.foliumap as geemap
from google.colab import drive
import json
import os

import pandas as pd
from sklearn import ensemble
import numpy as np

import time

from math import sqrt

In [ ]:
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=1TZhPkfpTTARy34ZblJkR2MLFgXMeoEULYQhPDcQoQA&tc=owddjI79JUujGyo0YaGm4XgizbUJm3LcirZ2tZyr5LE&cc=8cBimARurOlJ9ONkbOIW7ulWp-8Q7GxqZAiEIbtLOaI

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfgeXvuzcoQeb8Hz5P6Yh_FGRg6lYBk8MjfYd9tIPx1qMMeXTf3ADS94FOE

Successfully saved authorization token.


In [ ]:
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
def geojson2FeatureCollection(path_file='/content/drive/MyDrive/fundar_deforestacion/ecoregiones/chaco_seco.geojson'):
  with open(path_file) as f:
    json_data = json.load(f)
  return(ee.FeatureCollection(json_data))

In [ ]:
def scale_factor(image):
  return image.expression('float(b("NDVI")*0.0001)') #//the value 0.0001 will be multiplied to all pixels of our ImageCollection

In [ ]:
def exporto_modis(area,
                  tipo_ndvi = "anual",
                  rango_anios = range(0, 4, 1),
                  file_sufix = "00-03"):

  lista_modis = []

  if tipo_ndvi == "anual":

    for i in rango_anios:

      if i < 10:
        
            selec_modis = modis.filterBounds(area).filterDate('200'+str(i)+'-01-01', '200'+str(i)+'-12-30').select('NDVI')
            selec_modis = selec_modis.map(scale_factor)
            selec_modis = selec_modis.median().rename('NDVI_200'+str(i)).clip(area)
            lista_modis.append(selec_modis)


      elif i>=10:

              selec_modis = modis.filterBounds(area).filterDate('20'+str(i)+'-01-01', '20'+str(i)+'-12-30').select('NDVI')
              selec_modis = selec_modis.map(scale_factor)
              selec_modis = selec_modis.median().rename('NDVI_20'+str(i)).clip(area)
              lista_modis.append(selec_modis)

    modis_filtrada = lista_modis[0]

    for images in lista_modis[1:]:
      modis_filtrada = modis_filtrada.addBands(images)

  elif tipo_ndvi == "cuatrimestral":
    
    for i in rango_anios:

      if i < 10:

        for t in range(1, 5):

          if t == 1: 

            selec_modis = modis.filterBounds(area).filterDate('200'+str(i)+'-01-01', '200'+str(i)+'-03-30').select('NDVI')
            selec_modis = selec_modis.map(scale_factor)
            selec_modis = selec_modis.median().rename('NDVI_200'+str(i)+'_'+str(t)).clip(area)
            lista_modis.append(selec_modis)
          
          elif t == 2: 

            selec_modis = modis.filterBounds(area).filterDate('200'+str(i)+'-04-01', '200'+str(i)+'-06-30').select('NDVI')
            selec_modis = selec_modis.map(scale_factor)
            selec_modis = selec_modis.median().rename('NDVI_200'+str(i)+'_'+str(t)).clip(area)
            lista_modis.append(selec_modis)

          elif t == 3: 
          
            selec_modis = modis.filterBounds(area).filterDate('200'+str(i)+'-07-01', '200'+str(i)+'-09-30').select('NDVI')
            selec_modis = selec_modis.map(scale_factor)
            selec_modis = selec_modis.median().rename('NDVI_200'+str(i)+'_'+str(t)).clip(area)
            lista_modis.append(selec_modis)

          elif t == 4: 
          
            selec_modis = modis.filterBounds(area).filterDate('200'+str(i)+'-10-01', '200'+str(i)+'-12-30').select('NDVI')
            selec_modis = selec_modis.map(scale_factor)
            selec_modis = selec_modis.median().rename('NDVI_200'+str(i)+'_'+str(t)).clip(area)
            lista_modis.append(selec_modis)

      elif i>10:

          for t in range(1, 5):

            if t == 1: 

              selec_modis = modis.filterBounds(area).filterDate('20'+str(i)+'-01-01', '20'+str(i)+'-03-30').select('NDVI')
              selec_modis = selec_modis.map(scale_factor)
              selec_modis = selec_modis.median().rename('NDVI_20'+str(i)+'_'+str(t)).clip(area)
              lista_modis.append(selec_modis)
            
            elif t == 2: 

              selec_modis = modis.filterBounds(area).filterDate('20'+str(i)+'-04-01', '20'+str(i)+'-06-30').select('NDVI')
              selec_modis = selec_modis.map(scale_factor)
              selec_modis = selec_modis.median().rename('NDVI_20'+str(i)+'_'+str(t)).clip(area)
              lista_modis.append(selec_modis)

            elif t == 3: 
            
              selec_modis = modis.filterBounds(area).filterDate('20'+str(i)+'-07-01', '20'+str(i)+'-09-30').select('NDVI')
              selec_modis = selec_modis.map(scale_factor)
              selec_modis = selec_modis.median().rename('NDVI_20'+str(i)+'_'+str(t)).clip(area)
              lista_modis.append(selec_modis)

            elif t == 4: 
            
              selec_modis = modis.filterBounds(area).filterDate('20'+str(i)+'-10-01', '20'+str(i)+'-12-30').select('NDVI')
              selec_modis = selec_modis.map(scale_factor)
              selec_modis = selec_modis.median().rename('NDVI_20'+str(i)+'_'+str(t)).clip(area)
              lista_modis.append(selec_modis)
          

    modis_filtrada = lista_modis[0]

    for images in lista_modis[1:]:
      modis_filtrada = modis_filtrada.addBands(images)

    # Python Implementation for Removing bands. Tiene que haber una manera de hacer que se borre la columna si tiene NA, averiguar más adelante
    nameOfBands = modis_filtrada.bandNames().getInfo()
    nameOfBands.remove("NDVI_2001_2")
    print(nameOfBands) #// Check if everything in order

    modis_filtrada = modis_filtrada.select(nameOfBands) # Select all bands except the one you wanna remove

  
  training = modis_filtrada.sampleRegions(**{
              'collection': train_set,
              'properties': ['label'],
              'scale': 250,
              'geometries':True
            })
  
  task_train = ee.batch.Export.table.toDrive(**{
          'collection': training,
          'description': 'ndvi_train_set_'+file_sufix,
          'folder':'fundar_deforestacion_input',
          'fileFormat': 'GeoJSON' })
          
  task_train.start()

  testing = modis_filtrada.sampleRegions(**{
              'collection': test_set,
              'properties': ['label'],
              'scale': 250,
              'geometries':True
            })

  task_test = ee.batch.Export.table.toDrive(**{
          'collection': testing,
          'description': 'ndvi_test_set_'+file_sufix,
          'folder':'fundar_deforestacion_input',
          'fileFormat': 'GeoJSON' })
          
  task_test.start()

  print("Exportando train y test de los años "+file_sufix)

In [ ]:
train_set = geojson2FeatureCollection('/content/drive/MyDrive/fundar_deforestacion/data/training_data.geojson')
test_set = geojson2FeatureCollection('/content/drive/MyDrive/fundar_deforestacion/data/test_data.geojson')

In [ ]:
modis = ee.ImageCollection("MODIS/MOD09GA_006_NDVI")
region =  ee.FeatureCollection('projects/ee-ldb-deforestacion-montes/assets/Chaco_Seco')

In [ ]:
landcover = ee.Image("projects/ee-germanrosati/assets/traj_7years_clust_recode")
landcover = landcover.select('b1')

In [ ]:
for i in [0, 3, 4, 7, 10, 13, 16, 18]:

  exporto_modis(region,
                tipo_ndvi = "anual",
                rango_anios = range(i, i+3, 1),
                file_sufix = str(i)+"-"+str(i+2))


Exportando train y test de los años 0-2
Exportando train y test de los años 3-5
Exportando train y test de los años 4-6
Exportando train y test de los años 7-9
Exportando train y test de los años 10-12
Exportando train y test de los años 13-15
Exportando train y test de los años 16-18
Exportando train y test de los años 18-20
